In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from drive.ML_Study import * # 让driver连接云盘的ML_Study

In [0]:
# For console
# 防断连：f12 复制到console里边运行 
function ClickConnect(){
  console.log("Working"); 
  document
    .querySelector("#top-toolbar > colab-connect-button")
    .shadowRoot
    .querySelector("#connect")
    .click()
}
 
setInterval(ClickConnect,60000)

In [9]:
pip install transformers

     |████████████████████████████████| 573kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 51.7MB/s 
     |████████████████████████████████| 3.7MB 43.9MB/s 
     |████████████████████████████████| 890kB 51.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=3c77a282629598fd2ba0cc378bcf319bd0767a2db4f404a1fba137a3058591e7
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
import json
import numpy as np
from tqdm import tqdm
import os
import tensorflow as tf
from transformers import *

In [0]:
# input最大长度
maxlen=128

In [0]:
# if 'COLAB_TPU_ADDR' not in os.environ:
#     print('ERROR: Not connected to a TPU runtime')
# else:
#     tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#     print ('TPU address is', tpu_address)

In [0]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [14]:
print('Lodaing BERT')
pretrainedModel_path = 'drive/ML_Study/bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(os.path.join(pretrainedModel_path, 'vocab.txt'))
config = BertConfig.from_json_file(os.path.join(pretrainedModel_path, 'config.json'))
print('==========END==========')

Lodaing BERT


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


==========END==========


In [0]:
# 生成新的SPO
def load_data(path):
    D = []
    with open(path, encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            D.append({
                'text': l['text'],
                'spo_list': [
                    (spo['subject'], (spo['predicate']+'_'+key), spo['object'][key])
                    for spo in l['spo_list'] for key in spo['object'] 
                ]
            })
    return D

# 得到s,p对应值的初始下标
def search(pattern, sequence):
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

# 寻找特殊符号的位置
def search_special_label(pattern, sequence):
    rt = []
    for i, sid in enumerate(sequence):
        if sid == pattern:
            rt.append(i)
    return rt

In [16]:
print('Loading Data')
# sample_train_data = load_data('drive/ML_Study/Baidu_ERE_Data/sample_data.json')
# train_data = load_data('drive/ML_Study/Baidu_ERE_Data/train_data.json')
valid_data = load_data('drive/ML_Study/Baidu_ERE_Data/dev_data.json')
# test_data = load_data('./data/example.test')
print('=========END==========')

Loading Data
=========END==========


In [0]:
# category mapping
def get_mapping(mapping_path):
    predicate2id, id2predicate = {}, {}
    with open(mapping_path, encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            for key in l['object_type']:
                p = (l['predicate']+'_'+key)
                if p not in predicate2id:
                    id2predicate[len(predicate2id)] = p
                    predicate2id[p] = len(predicate2id)
    return predicate2id, id2predicate
    
mapping_path = 'drive/ML_Study/Baidu_ERE_Data/schema.json'

predicate2id, id2predicate = get_mapping(mapping_path)

In [0]:
# get BIO-relation label mapping
# get predicate mapping
special_labels = ['[PAD]','[CLS]','[SEP]','[UNK]']
special_labels_token = [0,101,102,100]
# All label
labels = []
for l in predicate2id.keys():
    l = 'B-Sub-'+l
    labels.append(l)
for l in predicate2id.keys():
    l = 'B-Obj-'+l
    labels.append(l)
labels.append('I')
labels.append('O')
for l in special_labels:
    labels.append(l)
    
# Bilabel Mapping
id2BIO = {}
BIO2id = {}
for i,label in enumerate(labels):
    id2BIO[i] = label
    BIO2id[label] = i

In [0]:
# 这个func应该没用了

# 0 1 形式label转为 BIO 形式 
# 由于可能一个实体可能既是SUB又是OBJ 而且可能对应多种关系 故采用list的格式存储
def visualized_label(label,maxlen=128):
    visual_label =[]
    for i in range(maxlen):
        a=[]
        visual_label.append(a)
    text_p, category_p= np.where(label>0.5)
    for p in zip(text_p, category_p):
        visual_label[p[0]].append(id2BIO[p[1]])
    return visual_label

# print(visualized_abel(label))

In [0]:
# spo 转为 X 和 0 1 形式的 label Y
# label: dimensionX->sequencePosition, dimensionY->CategoryPosition
def get_input(data,tokenizer,predicate2id,maxlen=128):
    special_labels_token = [0,101,102,100]
    token_ids, segment_ids, position_ids= [], [], []
    spo_labels = []
    for d in data:
        # get X 
        token_id, segment_id, position_id = tokenizer.encode_plus(d['text'], max_length=maxlen,pad_to_max_length=True).values()
        token_ids.append(token_id)
        segment_ids.append(segment_id)
        position_ids.append(position_id)
        # get Y
        # 116 = 2R + 4 + 2, R=55
        # I:110, O:111
        label = np.zeros((116,maxlen)) # len(labels), maxlen
        label[111,:] = 1 # 先设置所有标签都是 O
        for s, p, o in d['spo_list']:
            s = tokenizer.encode(s)[1:-1]# 101 是 CLS 102 是ESP 这里只是为了得到s内部值的ID
            p_i = predicate2id[p] # 对category进行映射得到对应的ID
            o = tokenizer.encode(o)[1:-1] 
            s_i = search(s, token_id) # 对text进行映射
            o_i = search(o, token_id)
            # Sub
            label[p_i,s_i] = 1
            label[111,s_i] = 0
            for i in range(1,len(s)):
                s_I_i = s_i + i
                label[110,s_I_i] = 1
                label[111,s_I_i] = 0
            # Obj
            label[55+p_i,o_i] = 1
            label[111,o_i] = 0
            for i in range(1,len(o)):
                o_I_i = o_i + i
                label[110,o_I_i] = 1
                label[111,o_I_i] = 0
        # specail label
        for i, sp in enumerate(special_labels_token):
            sp_is = search_special_label(sp, token_id)
            for sp_i in sp_is:
                label[112+i,sp_i] = 1
                label[111,sp_i] = 0
                
        spo_labels.append(label.T)
    # return [
    #         tf.convert_to_tensor(tf.constant(token_ids)),
    #         tf.convert_to_tensor(tf.constant(position_ids)),
    #         tf.convert_to_tensor(tf.constant(segment_ids)),
    #         tf.convert_to_tensor(tf.constant(spo_labels))
    # ]
    return [token_ids, position_ids, segment_ids, spo_labels] 
    # return [{'input_ids':token_ids, 'attention_mask':position_ids, 'token_type_ids':segment_ids},spo_labels]

In [21]:
print('Contructing Data')
# sample_train_xy = get_input(sample_train_data,tokenizer,predicate2id,maxlen=128)
# train_xy = get_input(train_data,tokenizer,predicate2id,maxlen=128)
valid_xy = get_input(valid_data,tokenizer,predicate2id,maxlen=128)
print('==========END==========')

Contructing Data
==========END==========


In [0]:
# train_dataset = tf.data.Dataset.from_tensor_slices((train_xy[0], train_xy[1])) # 构建 tf.dataset
# train_dataset = train_dataset.shuffle(buffer_size=1000)
# train_dataset = train_dataset.batch(batch_size=16)
# train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
tfr_data_path_sample = r'drive/ML_Study/Baidu_ERE_Data/TFrecord_data/sample_train.tfrecords'
tfr_data_path_train = r'drive/ML_Study/Baidu_ERE_Data/TFrecord_data/train.tfrecords'
tfr_data_path_dev = r'drive/ML_Study/Baidu_ERE_Data/TFrecord_data/valid.tfrecords' 

In [0]:
# 转为tfrecords 
def generate_tfr(path,data):
  with tf.io.TFRecordWriter(tfr_data_path_dev) as writer:
    i = 0
    for token_id, position_id, segment_id, spo_label in zip(data[0],data[1],data[2],data[3]):
        i+=1
        if i%5000 == 0:
          print('The '+str(i)+'th examples')
        feature ={
            'input_ids' : tf.train.Feature(int64_list=tf.train.Int64List(value=token_id)),
            'attention_mask' : tf.train.Feature(int64_list=tf.train.Int64List(value=position_id)),
            'token_type_ids' : tf.train.Feature(int64_list=tf.train.Int64List(value=segment_id)),
            'label' : tf.train.Feature(float_list=tf.train.FloatList(value=spo_label.flatten())) # 2D numpy 处理
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())  
    print('==========END==========')

In [0]:
# generate_tfr(tfr_data_path_sample,sample_train_xy)
# generate_tfr(tfr_data_path,train_xy)
# generate_tfr(tfr_data_path_dev,valid_xy)

In [0]:
# decode
# sample_raw_dataset = tf.data.TFRecordDataset(tfr_data_path_sample) 
train_raw_dataset = tf.data.TFRecordDataset(tfr_data_path_train) 
valid_raw_dataset = tf.data.TFRecordDataset(tfr_data_path_dev)

feature_description = { # 定义Feature结构，告诉解码器每个Feature的类型是什么
    'input_ids' : tf.io.FixedLenFeature([128], tf.int64),
    'attention_mask' : tf.io.FixedLenFeature([128], tf.int64),
    'token_type_ids' : tf.io.FixedLenFeature([128], tf.int64),
    'label' : tf.io.FixedLenFeature((128,116), tf.float32)
}

def _read_and_decode(example_string): # 将 TFRecord 文件中的每一个序列化的 tf.train.Example 解码
    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    # token_ids = tf.cast(feature_dict['input_ids'], tf.int64)
    # attention_mask = tf.cast(feature_dict['attention_mask'], tf.int64)
    # token_type_ids = tf.cast(feature_dict['token_type_ids'], tf.int64)
    # label = tf.cast(feature_dict['label'], tf.float32)
    # return {'input_ids':token_ids, 'attention_mask':attention_mask, 'token_type_ids':token_type_ids}, label
    return {'input_ids':feature_dict['input_ids'], 'attention_mask':feature_dict['attention_mask'], 'token_type_ids':feature_dict['token_type_ids']}, feature_dict['label']

# sample_dataset = sample_raw_dataset.map(_read_and_decode)
# sample_dataset = sample_dataset.shuffle(buffer_size=1000)
# sample_dataset = sample_dataset.batch(batch_size=16)
# sample_dataset = sample_dataset.prefetch(tf.data.experimental.AUTOTUNE)


train_dataset = train_raw_dataset.map(_read_and_decode)
train_dataset = train_dataset.shuffle(buffer_size=1000)
train_dataset = train_dataset.batch(batch_size=32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

valid_dataset = valid_raw_dataset.map(_read_and_decode)
valid_dataset = valid_dataset.shuffle(buffer_size=1000)
valid_dataset = valid_dataset.batch(batch_size=32)
valid_dataset = valid_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
class SPO_Model(TFBertPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = TFBertMainLayer(config, name="bert")
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        # self.BiLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True))
        self.classifier = tf.keras.layers.Dense(116,activation=tf.nn.sigmoid,kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05))
    def call(self, inputs, **kwargs):
        bert_outputs = self.bert(inputs, **kwargs)
        sequence_output = bert_outputs[0]
        x = self.dropout(sequence_output, training=kwargs.get("training", False))
        # x = self.BiLSTM(x)
        x = self.classifier(x)
        x = tf.keras.layers.Lambda(lambda x: x**(1/0.95))(x) #对BinaryEntropy进行放大处理，以排除人工标注错误
        # CRF Layer 其实不适合该模型
        return x

In [0]:
def myloss(y_true, y_pred):
	return tf.reduce_sum(tf.keras.losses.binary_crossentropy(y_true,y_pred))

In [0]:
spo_model = SPO_Model.from_pretrained(pretrainedModel_path,config=config)

spo_model.compile(
           optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-5), 
           loss=myloss, 
           metrics=[tf.keras.metrics.BinaryAccuracy()]) # accuracy没有好的 随便选了一个

In [42]:
spo_model.summary()

Model: "spo__model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  102267648 
_________________________________________________________________
dropout_113 (Dropout)        multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  89204     
Total params: 102,356,852
Trainable params: 102,356,852
Non-trainable params: 0
_________________________________________________________________


In [0]:
# with strategy.scope(): # only available data stored in google storage
#   spo_model = SPO_Model.from_pretrained(pretrainedModel_path,config=config)
#   spo_model.summary()
#   spo_model.compile(
#            optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-5), 
#            loss=myloss, 
#            metrics=[tf.keras.metrics.BinaryAccuracy()]) # accuracy没有好的 随便选了一个

In [0]:
spo_model.fit(train_dataset, validation_data=valid_dataset, epochs=5)

Epoch 1/5


    711/Unknown - 1132s 2s/step - loss: 211.8177 - binary_accuracy: 0.9898

In [0]:
spo_model.fit(train_dataset, validation_data=valid_dataset, epochs=5)

In [0]:
spo_model.save_weights('drive/ML_Study/saved_Model/weight_200413/spo_model_10epochs.ckpt')

In [0]:
spo_model.fit(train_dataset, validation_data=valid_dataset, epochs=5)

In [0]:
spo_model.fit(train_dataset, validation_data=valid_dataset, epochs=5)

In [0]:
spo_model.save_weights('drive/ML_Study/saved_Model/weight_200413v2/spo_model_20epochs.ckpt')

In [0]:
# spo_model.load_weights('drive/ML_Study/saved_Model/weight_200411/spo_model_10epochs.ckpt')

In [0]:
a = valid_data[3]['text']
b = valid_data[3]['spo_list']
print(a)
print(b)

这件婚事原本与陈国峻无关，但陈国峻却“欲求配而无由，夜间乃潜入天城公主所居通之
[('天城公主', '丈夫_@value', '国峻'), ('国峻', '妻子_@value', '天城公主')]


In [0]:
a_token = tokenizer.encode(a, max_length=128, pad_to_max_length=True)
print(a_token)

[101, 6821, 816, 2042, 752, 1333, 3315, 680, 7357, 1744, 2295, 3187, 1068, 8024, 852, 7357, 1744, 2295, 1316, 100, 3617, 3724, 6981, 5445, 3187, 4507, 8024, 1915, 7313, 718, 4052, 1057, 1921, 1814, 1062, 712, 2792, 2233, 6858, 722, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:
label = spo_model(tf.constant([a_token]))
print(label)

tf.Tensor(
[[[6.12396525e-06 9.75153944e-06 2.46941227e-07 ... 9.99919415e-01
   1.18960770e-06 1.70931817e-05]
  [4.91883213e-07 3.84611167e-06 6.18514449e-08 ... 1.87144406e-06
   5.94947691e-09 1.99374242e-08]
  [2.18663018e-07 7.74132161e-07 3.71924855e-08 ... 1.19686465e-08
   1.55069446e-09 4.32299707e-09]
  ...
  [2.96188176e-07 2.02725664e-06 2.06809627e-08 ... 7.87636356e-09
   2.08752802e-08 1.36730546e-06]
  [8.83167900e-07 4.63495053e-06 7.38263211e-08 ... 5.67632652e-09
   7.13860047e-08 5.96403243e-06]
  [1.10422534e-05 4.13609087e-05 1.59816670e-06 ... 3.09905901e-09
   6.09052131e-06 1.65553036e-04]]], shape=(1, 128, 116), dtype=float32)


In [0]:
rt = predict_SPOes(a_token, label[0])
print(rt)

[('陈 国 峻', '妻子_@value', '天 城 公 主'), ('天 城 公 主', '丈夫_@value', '陈 国 峻')]


In [0]:
# loss, acc = m1.evaluate(input, label)

In [0]:
# label -> SPOes
def predict_SPOes(token_id, label): # zip(token_id, label_pred)
  SPOes=[]
  sub_text_p, sub_category_p = np.where(label[:,:55]>0.75) 
  obj_text_p, obj_category_p = np.where(label[:,55:110]>0.65) # obj_category_p 自动减去48!! 返回的值是从 0 -> 47!
  i_text_p = sorted(np.where(label[:,110]>0.5)) # I 的对应下标
  sub_info = list(zip(sub_text_p, sub_category_p))
  obj_info = list(zip(obj_text_p, obj_category_p))

  SPOes=[]

  for sub in sub_info:
    sub_start = sub[0]
    sub_end = sub_start
    for I_i in i_text_p[0]: # 确定 sub 的长度
        if I_i == (sub_end + 1):
            sub_end += 1
    for obj in obj_info: 
        if sub[1] == (obj[1]):
            obj_start = obj[0]
            obj_end =  obj_start
            for I_i in i_text_p[0]: # 确定 obj 的长度
                if I_i == (obj_end + 1):
                    obj_end += 1
            spo = (
                tokenizer.decode(token_id[sub_start: sub_end+1]),
                id2predicate[sub[1]],
                tokenizer.decode(token_id[obj_start: obj_end+1])
                )
            SPOes.append(spo)
  return SPOes

In [0]:
def get_relation_pattern(mapping_path):
    relation_pattern = {}
    with open(mapping_path,encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            relation_pattern[l['predicate']] = (l['object_type'],l['subject_type'])
    return relation_pattern

relation_pattern = get_relation_pattern(mapping_path)

In [0]:
# SPOes -> final output
def get_spo_list(SPOes,relation_pattern):
    complex_relations = ['配音','上映时间','票房','获奖','饰演']
    spo_list= []
    for i1, spo in enumerate(SPOes):
        p = spo[1].split('_') # 切开predicate
        if p[1]  == '@value':
            obj={}
            obj[p[1]] = spo[2]
            obj_type = {}
            obj_type[p[1]] = relation_pattern[p[0]][0][p[1]] # object_type
            if p[0] in complex_relations: # 如果是复杂关系就找其子预测       
                for i2, spo_c in enumerate(SPOes):
                    if i1 == i2:
                        continue
                    p_c = spo_c[1].split('_')
                    if p_c[0] == p[0]:
                        obj[p_c[1]] = spo_c[2]
                        obj_type[p_c[1]] = relation_pattern[p[0]][0][p_c[1]] # object_type
        
            spo_list.append({
                        'predicate':p[0],
                        'object_type':obj_type,# 只保留存在的key
                        'subject_type':relation_pattern[p[0]][1],
                        'object':obj,
                        'subject':spo[0]
            })
    return spo_list